In [2]:
import re
import logging
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.types import DATE, VARCHAR, BigInteger
from urllib.parse import quote_plus

# ─── CONFIGURATION ─────────────────────────────────────────────────────────
SERVER     = "157.119.230.120,4071"
DATABASE   = "PDPSearchEcom_Tableau"
USER       = "ecomm"
PASSWORD   = "Hs%6uCx"
DRIVER     = "ODBC Driver 17 for SQL Server"
SCHEMA     = "dbo"
SRC_TABLE  = "Orient_PDP_Amazonv1"
TGT_TABLE  = "OrientPDPDatav1"

# ─── FINAL COLUMN ORDER (from your M-code) ──────────────────────────────────
EXPECTED_COLS = [
    "PDPSearchId","Platform","Date","ReportFreqTime","Pincode","City",
    "Product Code","Product Name","Model Name","Brand Name",
    "Product Type","Product Category","MRP","Sales Price",
    "Discount Value","Discount %","Pack Size","Availability",
    "Category","Category Rank","No of Sellers","Seller Name",
    "Seller Type","Total Rating","Average Rating","Promo Status",
    "MOP","Units Model Level (M)","List_SKU_per_Model_Name",
    "Per_SKU_Unit_Contribution","PincodeContribution",
    "Delivery Type","Delivery Date","TTD","Delivery Status",
    "Is_Last2Days","MOP Variance","MOP Flag","Competition Code"
]

# ─── LOGGING ─────────────────────────────────────────────────────────────────
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)5s | %(message)s",
    datefmt="%H:%M:%S"
)
log = logging.getLogger()

# ─── 1️⃣ Connect & Inspect ───────────────────────────────────────────────────
log.info("1️⃣ Connecting to database")
pw_esc = quote_plus(PASSWORD)
url    = f"mssql+pyodbc://{USER}:{pw_esc}@{SERVER}/{DATABASE}?driver={DRIVER.replace(' ', '+')}"
engine = create_engine(url)
insp   = inspect(engine)

# ─── 2️⃣ Watermark for incremental load ──────────────────────────────────────
log.info("2️⃣ Fetching last PDPSearchId for Platform='Amazon'")
last_id = 0
if insp.has_table(TGT_TABLE, schema=SCHEMA):
    cols = [c["name"] for c in insp.get_columns(TGT_TABLE, schema=SCHEMA)]
    if "PDPSearchId" in cols:
        with engine.connect() as conn:
            last_id = conn.execute(
                text(f"SELECT MAX(PDPSearchId) FROM {SCHEMA}.{TGT_TABLE} WHERE Platform='Amazon'")
            ).scalar() or 0
        log.info(f"   ✔ Watermark = {last_id}")
    else:
        log.warning("   ⚠ PDPSearchId missing; full load")
else:
    log.info("   ℹ Target not found; full load")

# ─── 3️⃣ Extract source table ─────────────────────────────────────────────────
log.info("3️⃣ Reading source table")
df = pd.read_sql_table(SRC_TABLE, engine, schema=SCHEMA)
log.info(f"   ✔ {len(df)} rows read")

# ─── 4️⃣ Incremental filter ───────────────────────────────────────────────────
log.info("4️⃣ Applying incremental filter")
df.rename(columns={
    "AmazonPDPSearchId":"PDPSearchId",
    "SellerName1":"Seller Name"
}, inplace=True)
df["PDPSearchId"] = pd.to_numeric(df["PDPSearchId"], errors="coerce")
df = df[df["PDPSearchId"] > last_id]
log.info(f"   ✔ {len(df)} rows with PDPSearchId > {last_id}")

# ─── 5️⃣ Parse Date ──────────────────────────────────────────────────────────
log.info("5️⃣ Parsing Date (DD-MM-YYYY)")
df["Date"] = pd.to_datetime(
    df["Date"], format="%d-%m-%Y", dayfirst=True, errors="coerce"
).dt.date
log.info("   ✔ Date parsed")

# ─── 6️⃣ Platform & text cleanup ─────────────────────────────────────────────
log.info("6️⃣ Setting Platform & cleaning text")
df["Platform"] = "Amazon"
def clean_text(col):
    return (col.astype(str)
               .str.strip()
               .replace(r"[^\x20-\x7E]","",regex=True))
for c in ["SalesPrice","MRP","DeliveryType1","Seller Name"]:
    if c in df:
        df[c] = clean_text(df[c]).replace("Currently unavailable","")
if "ClientName" in df:
    df.drop(columns="ClientName", inplace=True)
log.info("   ✔ Text cleaned")

# ─── 7️⃣ Numeric conversions ─────────────────────────────────────────────────
log.info("7️⃣ Stripping commas & converting numeric")
num_cols = [
    "SalesPrice","MRP","Category Rank","Total Rating","Average Rating",
    "No of Sellers","MOP","Units Model Level (M)","Per_SKU_Unit_Contribution",
    "PincodeContribution"
]
for c in num_cols:
    if c in df:
        s = df[c].astype(str).str.replace(",", "", regex=False).replace("", pd.NA)
        df[c] = pd.to_numeric(s, errors="coerce")
log.info("   ✔ Numeric conversion done")

# ─── 8️⃣ Compute MRPa & Sales Price ─────────────────────────────────────────
log.info("8️⃣ Computing MRPa & Sales Price")
def hi(a,b):
    if pd.isna(a) and pd.isna(b): return pd.NA
    if pd.isna(b): return a
    if pd.isna(a): return b
    return max(a,b)
def lo(a,b):
    if pd.isna(a) and pd.isna(b): return pd.NA
    if pd.isna(b): return a
    if pd.isna(a): return b
    return min(a,b)
df["MRPa"]        = df.apply(lambda r: hi(r["MRP"], r["SalesPrice"]), axis=1)
df["Sales Price"] = df.apply(lambda r: lo(r["MRP"], r["SalesPrice"]), axis=1)
log.info("   ✔ MRPa & Sales Price computed")

# ─── 9️⃣ Discount metrics ───────────────────────────────────────────────────
log.info("9️⃣ Calculating Discount Value & Discount % (2 decimals)")
df["Discount Value"] = df["MRPa"] - df["Sales Price"]
df["Discount %"]     = ((df["Discount Value"] / df["MRPa"]) * 1.0).round(2)
log.info("   ✔ Discounts calculated")

# 🔟 Add missing columns per M-code
log.info("🔟 Computing Category Rank, No of Sellers, Units Model Level (M), PincodeContribution, MOP Variance & MOP Flag")
# 1) rename &/or compute
if "ProductRank" in df:
    df["Category Rank"] = pd.to_numeric(df["ProductRank"], errors="coerce")
if "NoofSellers" in df:
    df["No of Sellers"] = pd.to_numeric(df["NoofSellers"], errors="coerce")
if "Units" in df:
    df["Units Model Level (M)"] = pd.to_numeric(df["Units"], errors="coerce")
if "PincodeContributionPer" in df:
    df["PincodeContribution"] = pd.to_numeric(df["PincodeContributionPer"], errors="coerce").round(2)
# 2) MOP Variance
df["MOP Variance"] = df["Sales Price"] - df["MOP"]
# 3) MOP Flag
def mop_flag(x):
    if pd.isna(x): return "Null"
    if x >= 99:    return "High"
    if x == 0:     return "Okay"
    if x <= -99:   return "Low"
    return "Medium"
df["MOP Flag"] = df["MOP Variance"].apply(mop_flag)
log.info("   ✔ Missing columns computed")

# ─── 1️⃣1️⃣ Delivery Date logic ──────────────────────────────────────────────
log.info("1️⃣1️⃣ Parsing Delivery Date (null when blank)")
mask_blank = (
    df["DeliveryDate1"].isna()
    | df["DeliveryDate1"].astype(str).str.strip().eq("")
)
df.loc[mask_blank, "Delivery Date"] = pd.NaT

df_nb = df.loc[~mask_blank].copy()
df_nb["TD1a"] = clean_text(df_nb["DeliveryDate1"].astype(str).str.partition(",")[2])
df_nb["TD1b"] = clean_text(df_nb["DeliveryDate1"].astype(str).str.partition("-")[2])
current_year = datetime.now().year

def parse_raw(r):
    raw = r["TD1b"] or r["TD1a"]
    try:
        return pd.to_datetime(f"{raw} {current_year}", dayfirst=True).date()
    except:
        return pd.NaT

df_nb["Parsed"] = df_nb.apply(parse_raw, axis=1)

def final_del(r):
    if pd.notna(r["Parsed"]):
        return r["Parsed"]
    m = re.search(r"-(\d+)\s*days", str(r.get("ShipmentDetail","")))
    if m:
        try:
            return r["Date"] + timedelta(days=int(m.group(1)) + 2)
        except:
            return pd.NaT
    return pd.NaT

df_nb["Delivery Date"] = df_nb.apply(final_del, axis=1)
df.update(df_nb[["Delivery Date"]])
df.drop(columns=["DeliveryDate1"], errors="ignore", inplace=True)
log.info("   ✔ Delivery Date computed")

# ─── 1️⃣2️⃣ TTD & status flags ───────────────────────────────────────────────
log.info("1️⃣2️⃣ Computing TTD & status flags")
df["TTD"] = (
    pd.to_datetime(df["Delivery Date"]) 
    - pd.to_datetime(df["Date"])
).dt.days
df["Delivery Status"] = df["TTD"].apply(
    lambda x: pd.NA if pd.isna(x) else ("Delay" if x > 3 else "On Time")
)
df["Is_Last2Days"] = df["Date"].apply(
    lambda d: "Last 2 Days"
    if d >= datetime.now().date() - timedelta(days=2)
    else "Old"
)
log.info("   ✔ TTD & flags set")

# ─── 1️⃣3️⃣ Delivery Type extraction ────────────────────────────────────────
log.info("1️⃣3️⃣ Extracting Delivery Type")
df["Delivery Type"] = df.apply(
    lambda r: (lambda s,p: pd.NA if s == str(p) else s)(
        re.sub(r"[A-Za-z]", "", str(r.get("DeliveryType1",""))).strip(),
        r.get("Pincode","")
    ),
    axis=1
)
log.info("   ✔ Delivery Type extracted")

# ─── 1️⃣4️⃣ Time bucketing ──────────────────────────────────────────────────
log.info("1️⃣4️⃣ Bucketing Times Of Day")
ptime = pd.to_datetime(df["Times Of Day"], format="%I:%M %p", errors="coerce")
df["Times Of Day"] = (
    pd.to_timedelta(ptime.dt.hour,   unit="h") 
    + pd.to_timedelta(ptime.dt.minute, unit="m")
)
def bucket(t):
    if pd.isna(t): return pd.NaT
    if   t < timedelta(hours=9):  return timedelta(hours=9)
    elif t < timedelta(hours=15): return timedelta(hours=15)
    elif t < timedelta(hours=18): return timedelta(hours=18)
    return timedelta(hours=21)

df["ReportFreqTime"] = df["Times Of Day"].apply(bucket)

def to_ampm(td):
    if pd.isna(td): return None
    secs = int(td.total_seconds()); h,rem = divmod(secs,3600); m,_ = divmod(rem,60)
    period = "AM" if h < 12 else "PM"; h12 = h % 12 or 12
    return f"{h12:02d}:{m:02d} {period}"

df["ReportFreqTime"] = df["ReportFreqTime"].apply(to_ampm)
log.info("   ✔ ReportFreqTime assigned")

# ─── 1️⃣5️⃣ SellerData join ─────────────────────────────────────────────────
log.info("1️⃣5️⃣ Joining SellerData")
if insp.has_table("SellerData", schema=SCHEMA):
    df["Seller Name"] = df["Seller Name"].str.title().str.strip()
    soldf = pd.read_sql_table("SellerData", engine, schema=SCHEMA)
    df = df.merge(
        soldf[["Seller Name","Seller Type"]],
        how="left", on="Seller Name"
    )
    df["Seller Type"].fillna("UnAuthorised", inplace=True)
else:
    df["Seller Type"] = "UnAuthorised"
log.info("   ✔ Seller Type set")

# ─── 1️⃣6️⃣ Final rename & reorder ──────────────────────────────────────────
log.info("1️⃣6️⃣ Finalizing columns")
df.rename(columns={
    "ProductCode":"Product Code",
    "Product_Name":"Product Name",
    "Model_Name":"Model Name",
    "Brand_Name":"Brand Name",
    "Product_Type":"Product Type",
    "Product_Category":"Product Category",
    "Pack_Size":"Pack Size",
    "Promo_Status":"Promo Status",
    "Competition_Code":"Competition Code"
}, inplace=True)
df = df[[c for c in EXPECTED_COLS if c in df.columns]]
log.info("   ✔ Columns ordered")

# ─── 1️⃣7️⃣ Schema enforcement & load ───────────────────────────────────────
log.info("1️⃣7️⃣ Verifying target schema & loading")
create_new = True
if insp.has_table(TGT_TABLE, schema=SCHEMA):
    actual = [c["name"] for c in insp.get_columns(TGT_TABLE, schema=SCHEMA)]
    if actual == EXPECTED_COLS:
        create_new = False
    else:
        log.warning("   ⚠ Schema mismatch; dropping existing table")
        with engine.begin() as conn:
            conn.execute(text(f"DROP TABLE {SCHEMA}.{TGT_TABLE}"))

dtype_map = {
    "PDPSearchId":    BigInteger(),
    "Date":           DATE(),
    "Delivery Date":  DATE(),
    "ReportFreqTime": VARCHAR(8)
}

if create_new:
    log.info("   ℹ Creating new target table")
    df.to_sql(
        TGT_TABLE, engine, schema=SCHEMA,
        index=False, dtype=dtype_map
    )
    log.info("   ✔ Table created & loaded")
else:
    log.info("   ℹ Appending to existing table")
    df.to_sql(
        TGT_TABLE, engine, schema=SCHEMA,
        if_exists="append", index=False
    )
    log.info("   ✔ Rows appended")

log.info("✅ ETL run complete")


22:38:32 |  INFO | 1️⃣ Connecting to database
22:38:32 |  INFO | 2️⃣ Fetching last PDPSearchId for Platform='Amazon'
22:38:32 |  INFO |    ✔ Watermark = 207126
22:38:32 |  INFO | 3️⃣ Reading source table
22:39:00 |  INFO |    ✔ 57325 rows read
22:39:00 |  INFO | 4️⃣ Applying incremental filter
22:39:00 |  INFO |    ✔ 2208 rows with PDPSearchId > 207126
22:39:00 |  INFO | 5️⃣ Parsing Date (DD-MM-YYYY)
22:39:00 |  INFO |    ✔ Date parsed
22:39:00 |  INFO | 6️⃣ Setting Platform & cleaning text
22:39:00 |  INFO |    ✔ Text cleaned
22:39:00 |  INFO | 7️⃣ Stripping commas & converting numeric
22:39:00 |  INFO |    ✔ Numeric conversion done
22:39:00 |  INFO | 8️⃣ Computing MRPa & Sales Price
22:39:00 |  INFO |    ✔ MRPa & Sales Price computed
22:39:00 |  INFO | 9️⃣ Calculating Discount Value & Discount % (2 decimals)
22:39:00 |  INFO |    ✔ Discounts calculated
22:39:00 |  INFO | 🔟 Computing Category Rank, No of Sellers, Units Model Level (M), PincodeContribution, MOP Variance & MOP Flag
22:3